# Receive

In [ ]:
from PIL import Image
import numpy as np
import socket
import struct
import matplotlib.pyplot as plt

In [ ]:
import itertools


def byte_array_to_bool_array(byte_array, original_shape):
    flat_bool_array = []
    
    for byte in byte_array:
        for bit in range(8):
            flat_bool_array.append(bool(byte & (1 << bit)))

    # flat_bool_array = flat_bool_array[:original_shape[0] * original_shape[1]]
    bool_array = np.array(flat_bool_array).reshape(original_shape)

    return bool_array

def plot_bool_array(bool_array):
    """
    Plot a 2D boolean array using matplotlib.
    
    :param bool_array: 2D numpy array of booleans
    """
    plt.imshow(bool_array, cmap='gray', interpolation='nearest')
    plt.title('2D Boolean Array')
    plt.show()

def receive(ip_address, port, buffer_size=1024):
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.bind((ip_address, port))
    print(f"Listening on {ip_address}:{port}")

    partial_data = {}
    array_shapes = {}

    while True:
        packet, addr = sock.recvfrom(buffer_size)
        print(len(packet))

        array_id = packet[0]
        seq_num = packet[1]
        num_packets = packet[2]
        width = packet[3]
        height = packet[4]
        data = packet[5:]

        if array_id not in partial_data:
            partial_data[array_id] = [None] * num_packets
            array_shapes[array_id] = (height, width)

        partial_data[array_id][seq_num] = data
        print(f"Received packet {seq_num + 1}/{num_packets} of array {array_id} from {addr}")

        if None not in partial_data[array_id]:
            full_byte_array = bytearray(itertools.chain(*partial_data[array_id]))
            original_shape = array_shapes[array_id]
            bool_array = byte_array_to_bool_array(full_byte_array, original_shape)
            plot_bool_array(bool_array)
            del partial_data[array_id]
            del array_shapes[array_id]


In [ ]:
IP_ADDRESS = "192.168.1.8"  # Listen on all interfaces
PORT = 57

receive(IP_ADDRESS, PORT)

## Send and receive

In [16]:
import socket
import struct

# Server details
HOST = '192.168.1.8'  # Replace with the IP address of your Arduino
PORT = 12345

# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

try:
    # Connect to the server
    sock.connect((HOST, PORT))
    print("[INFO] Connected to server")

    # Prepare sample data to send (display buffer)
    # For example, we'll send a simple pattern (1024 bytes)
    display_buffer = bytearray([0xFF if i % 2 == 0 else 0x00 for i in range(1024)])

    # Send data
    sock.sendall(display_buffer)
    print("[INFO] Sent data to server")

    # Receive response (8 floats, 4 bytes each, 32 bytes total)
    reply_buffer = sock.recv(8 * 4)

    # Unpack the received data into floats
    data = struct.unpack('8f', reply_buffer)

    # Print the received data
    battery_level = data[0]
    temperature = data[1]
    ax = data[2]
    ay = data[3]
    az = data[4]
    gx = data[5]
    gy = data[6]
    gz = data[7]

    print(f"Battery Level: {battery_level}%")
    print(f"Temperature: {temperature}°C")
    print(f"Acceleration X: {ax} m/s^2, Y: {ay} m/s^2, Z: {az} m/s^2")
    print(f"Gyroscope X: {gx} rad/s, Y: {gy} rad/s, Z: {gz} rad/s")

finally:
    # Close the socket
    sock.close()
    print("[INFO] Connection closed")


[INFO] Connected to server
[INFO] Sent data to server
Battery Level: 83.59375%
Temperature: 32.029998779296875°C
Acceleration X: 0.4908113479614258 m/s^2, Y: 0.07422025501728058 m/s^2, Z: 8.564059257507324 m/s^2
Gyroscope X: -0.08260336518287659 rad/s, Y: -0.0005329249543137848 rad/s, Z: -0.0013323123566806316 rad/s
[INFO] Connection closed
